<a href="https://colab.research.google.com/github/aRod209/pytorch-for-deep-learning/blob/main/lectures/06_pytorch_transfer_learning_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06. PyTorch Transfer Learning

What is transfer learning?

Transfer learning involves taking the parameters of what one model has learned on another dataset and applying to our own problem.

* Pretrained model = foundation model

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.2.1+cu121
0.17.1+cu121


In [3]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
# I know we have versions that are greater than these versions, but I just
# want to run the below code anyways to know how to update to targeted versions.

try:
  assert int(torch.__version__.split('.')[0]) >= 1, 'torch version should be at least version 1'

  if int(torch.__version__.split('.')[0]) == 1:
    assert int(torch.__version__.split('.')[1]) >= 12, 'torch version should be 1.12+'

  if int(torchvision.__version__.split('.')[0]) == 0:
    assert int(torchvision.__version__.split('.')[1]) >= 13, 'torchvision version should be 0.13+'
except:
  print(f'[INFO] torch/torchvision version not as required, installing nightly versions.')
  !pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
finally:
  print(f'torch version: {torch.__version__}')
  print(f'torchvision version: {torchvision.__version__}')

torch version: 2.2.1+cu121
torchvision version: 0.17.1+cu121
